In [7]:
import numpy as np
# Define A, B, C matrices
A = np.array([[1, 0.1], [0, 1]])
B = np.array([[0.005] , [0.1]])
C = np.array([1 , 0])
# Known variable values
h = 0.1
K = np.array([16.0302 , 5.6622])
L = np.array([[1.8721], [9.6532]])
r_min = 0.51
Th = 0.003
# Unknown values
y = 0; # sensor limit
u = 0; # Saturation limit
# Attack vector a
temp_1=[x*1000 for x in [0.2570,    0.0068,    0.0011 ,  -0.0021  , -0.0034,   -0.0100 ,   0.0032 ,  -0.0025,   -0.0018  ,  1.123]]
temp_2=[-1.2822  , -3.8838,   -6.5325 ,  -9.1790 , -11.8007,  -14.3584,  -16.8851 , -19.4185,  -21.9335,  -30.0617]
# temp_1=[-245.623734373287,	-3.86574806140351,	-2.36899303107162,	-1.26643271870318,	-0.543203915125821,	-0.130860356864007,	0.0556101393110584,	0.0961058733369649	,0.0550891047181291,	0.486766522911399,	-1199.83099881780]
# temp_2=[1.22511867186643,	3.69217249590630,	6.18750406540856,	8.69811680365968,	11.2148817650800,	13.7321210878602,	16.2468407017282,	18.7579057755329,	21.2713189144472,	23.7841513352235,	32.2855763574743]
# Known values for the second row
t = len(temp_1)

temp_1.insert(0,0)
temp_1.append(0)

temp_2.insert(0,0)
temp_2.append(0)


#print(temp_1)
a = np.zeros((3, t+1+1))  #The last column is also padded with zeros
#a_u is the first row, a_y is the second row
# Assign the known values to the second row (index 1)
a[1, :] = temp_1
a[2, :] = temp_2
#print(a)

In [8]:
def AdvPatSyn(A,B,C,K,L,y,u,Th,r_min,a):
    D = np.zeros((t + 1, t + 1))
    subPatternList = []
    #count = 0
    #delta_e = np.zeros((2, 1))
    delta_e = []
    delta_e.append([[0], [0]])
    delta_r = [a[2][1]]

    #May be an error
    rho = np.array([[np.zeros(t+1) for _ in range(t+1)] for _ in range(t+1)])  #t+1 * t+1 matrix, with a sequence of t+1 values

    # ResDiffGen function
    for k in range(1, t + 1):    #it will loop from 1 to t
        delta_e.append([[0],[0]])
        delta_r.append(0)
        delta_r[k] = (C @ A @ delta_e[k-1]) + (C @ B) * a[1][k] + a[2][k+1]
        for i in range(k):  #it will loop from 0 to k-1
            
            delta_e[k] = delta_e[k] + ((np.linalg.matrix_power(A, i)) @ ((B * a[1][k-1-i]) - (L * delta_r[k-i])))
        
    
    for k in range(1,t+1):
        for l in range(1,t-k+1):
            lhs = np.zeros((2, 1))
            rhs = np.zeros((2, 1))
            rho[k][l] = np.ones(t+1)
            for i in range(1, l+1):
                if k > 1:
                    lhs = lhs + ((np.linalg.matrix_power(A, i-1)) @ B * (a[1][k+l-i] - a[1][k-1]))
                else:
                    lhs = lhs + ((np.linalg.matrix_power(A, i-1)) @ B * a[1][k+l-i])
                rhs = rhs + ((np.linalg.matrix_power(A, i-1)) @ L * delta_r[k+l-i+1])
            if np.linalg.norm(lhs) > np.linalg.norm(rhs):
                # to make the sequence of 1^k 0^l-k....
                if (l>k):
                    start_zero = np.zeros(1)
                    ones_start = np.ones(k)
                    zeros_middle = np.zeros(l-k)
                    ones_end = np.ones(t-l)

                    # Concatenate the arrays
                    rho[k][l] = np.concatenate((start_zero, ones_start, zeros_middle, ones_end))

                if np.sum(rho[k][l]) >= r_min * t :
                    D[k][l] = np.linalg.norm(lhs) - np.linalg.norm(rhs)
                    temp_matrix = rho[k][l][1:]
                    subPatternList.append(temp_matrix)
                else:
                    rho[k][l] = np.ones(t+1)
            



    return D, subPatternList

In [9]:
def converter(Y):
    # If Y is a list, convert it to a NumPy array first
    if isinstance(Y, list):
        Y = np.array(Y)

    # Convert the float values (1.0 and 0.0) to integers (1 and 0)
    int_Y = Y.astype(int)

    # Convert each inner array to a string and return the result as an array of strings
    string_Y = np.array(["".join(map(str, row)) for row in int_Y])

    return string_Y

In [10]:
X, Y = AdvPatSyn(A,B,C,K,L,y,u,Th,r_min,a)
print(Y)
# '''
# print(count)
# print(Y)
# print(X)
# # X -- D matrix
# Y = converter(Y)
# print(Y)
# print(len(Y))
# print(len(X),len(X[0]))
# # Y -- subpattern list

subPatternList = []
for k in range(1,t+1):
    for l in range(1,t+1):
        if X[k][l] > 0:
            #print(X[k][l])
            subPatternList.append('1'*k + '0'*(l-k)+'1'*(t-l))
Y=subPatternList
print(subPatternList)
print(X)
        

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 0., 0., 1., 1., 1., 1., 1., 1., 1.]), array([1., 0., 0., 0., 0., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1.]), array([1., 1., 0., 0., 1., 1., 1., 1., 1., 1.]), array([1., 1., 0., 0., 0., 1., 1., 1., 1., 1.]), array([1., 1., 0., 0., 0., 0., 1., 1., 1., 1.])]
['1111111111', '1001111111', '1000011111', '11111111111', '1111111111', '1101111111', '1100111111', '1100011111', '1100001111']
[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [  0.          17.1243729    0.          17.69153812   0.
   19.37006396   0.           0.           0.           0.
    0.        ]
 [  0.           4.59633306  48.88758377  41.48847068  91.33045515
   72.86558005 116.41553377   0.           0.           0.
    0.        ]
 [  0.           0. 

In [11]:
def zero(s):
    zero_started = False
    for i in range(len(s)):
        if s[i] == '0':
            zero_started = True
        elif zero_started and s[i] == '1':
            return i
    # If the entire string is zeros after the first streak, return the length
    return len(s)

def merge_patterns(pattern1, pattern2):
    """
    Merge two binary patterns using bitwise AND, ensuring non-overlapping zeros.
    """
    return ''.join('1' if pattern1[k] == '1' and pattern2[k] == '1' else '0'
                   for k in range(len(pattern1)))

def one(s):
    one_started = False
    for i in range(len(s)):
        if s[i] == '1':
            one_started = True
        elif one_started and s[i] == '0':
            return i
    # If the entire string is ones after the first streak, return the length
    return len(s)

def rate(s):
    # Count the number of '1's in the string
    num_ones = s.count('1')
    # Calculate the total length of the string
    total_length = len(s)
    # Calculate and return the rate
    return num_ones / total_length if total_length > 0 else 0

def initialize_M(D):
    # Create a matrix M with the same dimensions as D
    rows = len(D)
    cols = len(D[0]) # Handle case where D is empty
    M = [[0 for _ in range(cols)] for _ in range(rows)]
    return M

def andd(s1, s2):
    # Ensure both strings are of the same length
    if len(s1) != len(s2):
        print(s1)
        print(s2)
        raise ValueError("Strings must be of the same length")

    # Perform bitwise AND and return the result as a string
    return ''.join('1' if s1[i] == '1' and s2[i] == '1' else '0' for i in range(len(s1)))

def g(x, y, z):
    # Generate the binary string based on x, y, z
    return '1' * x + '0' * y + '1' * z

def generate_P(D, s, rmin, end0, end1):
    """
    Generate the matrix P which stores the best control execution patterns up to each position.
    """
    # Initialize matrix P with the same dimensions as D
    # print(D)
    t = len(s[0])
    rows = len(s)
    cols = len(s[0])
    print("size:", rows,cols)
    P = [['0'*t for _ in range(cols+1)] for _ in range(rows+1)]
    M=np.zeros((rows+1,cols+1))
    # print(s)
    print("t=",t)
    # Handle the first sub-pattern (i = 1)
    for j in range(1,cols+1):
        if j < end0[1]:
            # Case for j < end0(1): P[1][j] = 1^t (all 1's)
            P[1][j] = '1' * t  # All execution (1's for length j+1)
            M[1][j]=0
        else:
            # Case for j >= end0(1): P[1][j] = 1^2 0 1^(j-3)
            if(rate(s[0][0:j])>=rmin):
              P[1][j] = '1' * end1[1] + '0' * (end0[1]-end1[1]) + '1' * (t - end0[1])
              M[1][j]=D[end1[1]][end0[1]]
            else:
              P[1][j] = '1' * t
              M[1][j]=0

    # Start filling matrix P for i > 1 (from second sub-pattern onwards)
    for i in range(2, rows+1):  # loop through sub-patterns starting from i=2
        for j in range(1,cols+1):  # loop through pattern positions
            # print("for j")
            # print(j,len(D[i]))
            if j < end0[i]:
                # Case: i > 1 and j < end0(i)
                P[i][j] = P[i-1][j]
                M[i][j]=M[i-1][j]
            else:

                # Check the rate of the j-length prefix
                if rate(s[i-1][0:j]) < rmin:
                    # If rate of the prefix is less than rmin, use the previous best pattern
                    M[i][j]=M[i-1][j]
                    P[i][j] = P[i-1][j]
                else:
                    # print(P[i][end1[i]-1])
                    # if(end1[i]-1>1):
                    s0 = andd(P[i][end1[i]-1], '1' * end1[i] + '0' * (end0[i] - end1[i]) + '1' * (t - end0[i]))
                    # print(s0)
                    # If the rate is >= rmin, we apply the merging rule
                    if rate(s0[0:j]) >= rmin:
                        if(M[i-1][j]>=M[i][end1[i]-1]+D[end1[i]][end0[i]]):
                            P[i][j] = P[i-1][j]
                            M[i][j]=M[i-1][j]
                        else:
                            P[i][j]=s0
                            M[i][j]=M[i][end1[i]-1]+D[end1[i]][end0[i]]
                    else:
                        # Otherwise, keep the previous best pattern
                        if(M[i-1][j]>=D[end1[i]][end0[i]]):
                            M[i][j]=M[i-1][j]
                            P[i][j] = P[i-1][j]
                        else:
                            M[i][j]=D[end1[i]][end0[i]]
                            P[i][j]= '1' * end1[i] + '0' * (end0[i] - end1[i]) + '1' * (t - end0[i])

    return P,M


def f(D,s,rmin):
    end0=[0]
    end1=[0]
    rates = [0]

    for i in range(len(s)):
        end0.append(zero(s[i]))
        end1.append(one(s[i]))
        rates.append(rate(s[i]))

    P,M = generate_P(D,s,rmin,end0,end1)

    return P,M

In [12]:

# t=8
# def lst(i,j):
  # return '1' * i + '0' * (j - i) + '1' * (t - j)

# values=[[2,3],[2,4],[2,5],[2,6],[3,4],[3,5],[3,6],[3,7],[4,5],[4,6],[4,7],[4,8],[5,6],[5,7],[5,8],[7,8]]
# Y= [lst(i,j) for (i,j) in values]
# X=[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0.02,0.08,0.15,3.7,0,0],[0,0,0,0,0.05,0.12,3.68,1.67,0],[0,0,0,0,0,0.06,3.6,1.64,1.77],[0,0,0,0,0,0,3.52,1.56,1.72],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0.08],[0,0,0,0,0,0,0,0,0]]
P,M = f(X,Y,0.51)
print(M)
print(P)



size: 9 10
t= 10
[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [  0.           0.           0.           0.           0.
   17.69153812  17.69153812  17.69153812  17.69153812  17.69153812
   17.69153812]
 [  0.           0.           0.           0.           0.
   17.69153812  17.69153812  17.69153812  17.69153812  19.37006396
   19.37006396]
 [  0.           0.           0.           0.           0.
   17.69153812  17.69153812  17.69153812  17.69153812  19.37006396
   19.37006396]
 [  0.           0.           0.           0.           0.
   17.69153812  17.69153812  17.69153812  17.69153812  19.37006396
   19.37006396]
 [  0.           0.           0.          41.48847068  41.48847068
   41.48847068  41.48847068  41.48847068  41.48847068  41.48847068
   41.4884706